In [1]:
import pandas as pd
import folium
import matplotlib.colors as mcolors
import branca.colormap as cm

In [2]:
# Load datasets
championship_data = pd.read_csv('Championship_data.csv', encoding = 'ISO-8859-1')
geo_data = pd.read_csv('uscities.csv')

# Clean city names in both datasets
championship_data['City'] = championship_data['City'].str.strip()
geo_data['city'] = geo_data['city'].str.strip()

# Align the State Name to avoid mismatch
# Maps championship data
city_mappings = {
    'Golden State': 'San Francisco',
    'Minnesota': 'Minneapolis',
    'Utah': 'Salt Lake City',
    'Indiana': 'Indianapolis'
}

championship_data['City'] = championship_data['City'].replace(city_mappings)

In [3]:
# Merge the datasets
geo_col = ['city', 'lat', 'lng']
champ_col = ['City', 'NBA Finals wins by team (1946 - 2024)', 'Number of Championship']
merged_data = pd.merge(
    geo_data[geo_col],
    championship_data[champ_col],
    left_on = 'city',
    right_on = 'City',
    how = 'inner'
)

In [4]:
def get_championship_color(championships):
    """
    Does:
        Get color by the number of championships each team wons.

    Parameters:
        championships: Int number of championships a team has won.

    Returns:
        A color name as a str, representing the championship range.

    Does:
    This function assigns a color to represent different ranges of championship counts:
    - Gray for 0 championships
    - Blue for 1-2 championships
    - Gold for 3-5 championships
    - Orange for 6-10 championships
    - Red for more than 10 championships
    """
    if championships == 0:
        return 'Gray'  
    elif championships <= 2:
        return 'Blue'  
    elif championships <= 5:
        return 'Gold'
    elif championships <= 10:
        return 'Orange'
    else:
        return 'Red'

In [5]:
# Create map filter the long and lat to US Territory
# Credit to https://www.geodatos.net/en/coordinates/united-states for the filter
interactive_map = folium.Map(location = [37.09, -95.71], zoom_start = 4)

In [6]:
# Add markers
for index, row in merged_data.iterrows():
    city = row['city']
    championships = int(row['Number of Championship'])
    team = row['NBA Finals wins by team (1946 - 2024)']
    latitude = row['lat']
    longitude = row['lng']
    
    color = get_championship_color(championships)
    
    popup_text = f"""
    <div style='font-family: Arial; font-size: 12px;'>
        <b>City:</b> {city}<br>
        <b>Team:</b> {team}<br>
        <b>Championships:</b> {championships}
    </div>
    """
    
    folium.CircleMarker(
        location = [latitude, longitude],
        radius = 12 if championships > 10 else\
                (9 if championships > 5 else (7 if championships > 0 else 5)),
        popup = folium.Popup(popup_text, max_width = 200),
        color = color,
        fill = True,
        fill_color = color,
        fill_opacity = 0.7,
        weight = 2
    ).add_to(interactive_map)

In [7]:
legend_html = '''
<div style="
    position: fixed;
    bottom: 50px;
    right: 50px;
    width: 150px;
    height: auto;
    background-color: white;
    border: 2px solid grey;
    z-index: 9999;
    border-radius: 6px;
    padding: 10px;
    font-family: Arial, sans-serif;
    font-size: 12px;
    box-shadow: 0 0 15px rgba(0,0,0,0.2);">
    <div style="text-align: center;"><b>NBA Championships</b></div>
    <hr style="margin: 5px 0;">
    <div style="margin: 5px 0;">
        <span style="color: Red; font-size: 20px; vertical-align: middle;">●</span>
        <span style="vertical-align: middle;"> >10 championships</span>
    </div>
    <div style="margin: 5px 0;">
        <span style="color: Orange; font-size: 18px; vertical-align: middle;">●</span>
        <span style="vertical-align: middle;"> 6-10 championships</span>
    </div>
    <div style="margin: 5px 0;">
        <span style="color: Gold; font-size: 16px; vertical-align: middle;">●</span>
        <span style="vertical-align: middle;"> 3-5 championships</span>
    </div>
    <div style="margin: 5px 0;">
        <span style="color: Blue; font-size: 14px; vertical-align: middle;">●</span>
        <span style="vertical-align: middle;"> 1-2 championships</span>
    </div>
    <div style="margin: 5px 0;">
        <span style="color: Gray; font-size: 12px; vertical-align: middle;">●</span>
        <span style="vertical-align: middle;"> 0 championships</span>
    </div>
</div>
'''

In [8]:
# Display the map
interactive_map.get_root().html.add_child(folium.Element(legend_html))
interactive_map